In [1]:
from io import StringIO
from pathlib import Path

from dapla_pseudo import PseudoData
import pandas as pd

JSON_FILE = "data/personer.json"
CSV_FILE = "data/personer.csv"

df = pd.read_json(
    JSON_FILE,
    dtype={
        "fnr": "string",
        "fornavn": "string",
        "etternavn": "string",
        "kjonn": "category",
        "fodselsdato": "string",
    },
)
df.head()

,fnr,fornavn,etternavn,kjonn,fodselsdato
0,02099510504,Donald,Duck,M,020995
1,06097048531,Mikke,Mus,M,060970
2,18099908978,Anton,Duck,M,180999


# Case: Single field default encryption (daead)

In [2]:
result = (
    PseudoData.from_pandas(df).on_field("fnr").pseudonymize()
)
result.dataframe.head()

KeyError: 'LOCAL_USER_PATH'

# Case: Single field sid mapping

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_field("fnr")
    .map_to_stable_id()
    .pseudonymize()
)
result.dataframe.head()

# Case: Single field FPE (used for e.g. existing stable ID/snr/sid)

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_field("fnr")
    .pseudonymize(preserve_formatting=True)
)
result.dataframe.head()

# Case: Multiple fields default encryption (daead)

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_fields("fornavn", "etternavn", "fodselsdato")
    .pseudonymize()
)
result.dataframe.head()

# Case: Chaining calls
Calls may simply be chained together to apply different pseudonymization to different fields.

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_field("fnr")
    .map_to_stable_id()
    .pseudonymize()
)
result = (
    PseudoData.from_pandas(result.dataframe)
    .on_fields("fornavn", "etternavn", "fodselsdato")
    .pseudonymize()
)
result.dataframe.head()